# Join - инструмент объединения данных из нескольких связанных таблиц

Реляционная модель данных подразумевает отдельное хранение и возможность независимой обработки данных для каждой сущности.

Вместе с тем, часто возникает потребность собрать данные из нескольких, связанных таблиц.

Как правило, сущности (таблицы) связаны друг с другом внешними связями по принципу (_primary key -- foreign key_).

Связи могут быть типа _"1 к 1"_ или _"1 ко многим"_ (с вариантами _"1 к 0 или 1"_, _"1 к 0 или более"_, _"1 к 2"_ и пр).<br/>
Связь _"многие-ко-многим"_ в реляционной модели обеспечивается с помощью дополнительной таблицы связей (другие названия: Link-таблица, Bridge-таблица, xref-таблица).

В зависимости от характера связей между таблицами, _логически операция соединения_ может быть:
- _внутренним соединением_ (INNER JOIN). При этом:
    - если условие соединения отсутствует, то такой INNER JOIN называют декартовым произведением (CROSS JOIN, CARTESIAN PRODUCT)
    - если для описания связи между наборами данных использются корреляционные подзапросы, то такой INNER JOIN называют CROSS APPLY
    
- _внешним соединением_ (OUTER JOIN). Разновидности - LEFT JOIN, RIGHT JOIN, FULL JOIN
    - если для описания связи между наборами данных использются корреляционные подзапросы, то такой OUTER JOIN называют OUTER APPLY



## Пример логической модели данных:

<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/SwimmersDB.png?raw=1" />

Обратите внимание на данные в наших таблицах. 

У некоторых спортсменов нет категории, клуба или тренера (или они неизвестны).<br/>
Встречаются спортсмены, которых ведёт несколько тренеров.

Про одни сущности есть чуть больше информации чем про другие.

In [19]:
use tempdb
go

--содержимое таблиц:
select * from dbo.SwimmingClub 
select * from dbo.Swimmer
select * from dbo.Category 

Commands completed successfully.

Total execution time: 00:00:00.0016185

(3 rows affected)

(7 rows affected)

(3 rows affected)

Total execution time: 00:00:00.0112856

SwimmingClubID Name City Address Phone YearEstablished ModifiedDate 2 ГЦОР Трактор Минск NULL NULL NULL 2019-04-29 12:40:51.780 3 ДЮСШ Янтарь Minsk NULL NULL NULL 2019-04-29 12:40:51.780 4 СК Олимпик-2011 Молодечно NULL NULL NULL 2019-04-29 12:40:51.780

SwimmerID FirstName LastName YearOfBirth Gender SwimmingClubID CategoryId ModifiedDate isdeleted 2 Анна Высоцкая 2007 ж 3 4 2019-04-29 12:40:51.810 1 3 Илья Гавриленко 2006 м 2 3 2019-04-29 12:40:51.810 0 4 Максим Кликушин 2007 м 2 3 2019-04-29 12:40:51.810 0 5 Матвей Данкевич 2006 м 4 2 2019-04-29 12:40:51.810 0 6 Никита Клюй 2005 м 3 3 2019-04-29 12:40:51.810 0 9 Gleb Bondarec 2007 м 2 NULL 2019-04-29 15:38:38.427 0 11 Алексей Рылько 2005 м 3 NULL 2019-04-30 10:58:20.420 1

CategoryID Name ModifiedDate 2 I 2019-04-29 12:40:51.767 3 II 2019-04-29 12:40:51.767 4 кмс 2019-04-29 12:40:51.767

<b>Пример #1</b>. Найти всех спортсменов из клуба Янтарь, имеющих II спортивный разряд.

Используя старую нотацию:

In [20]:
use tempdb
go

select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, c.[Name] Category
from dbo.SwimmingClub sc, dbo.Swimmer s, dbo.Category c
where sc.[Name] like N'%Янтарь%' 
      and sc.SwimmingClubID = s.SwimmingClubID
      and s.CategoryID     = c.CategoryID
      and c.[Name]         = N'II'

Commands completed successfully.

Total execution time: 00:00:00.0012785

(1 row affected)

Total execution time: 00:00:00.0048865

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II

Используя новую нотацию:

In [21]:
use tempdb
go

select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, c.[Name] Category
from dbo.SwimmingClub sc
inner join dbo.Swimmer s  on s.SwimmingClubID = sc.SwimmingClubID
inner join dbo.Category c on s.CategoryID     = c.CategoryID
where sc.[Name] like N'%Янтарь%' and c.[Name] = N'II'

Commands completed successfully.

Total execution time: 00:00:00.0020676

(1 row affected)

Total execution time: 00:00:00.0045204

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II

Используя CROSS JOIN:

In [30]:
use tempdb
go

select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, c.[Name] Category
from dbo.SwimmingClub sc
cross join dbo.Swimmer s
cross join dbo.Category c
where sc.[Name] like N'%Янтарь%' 
      and sc.SwimmingClubID = s.SwimmingClubID
      and s.CategoryID     = c.CategoryID
      and c.[Name]         = N'II'

Commands completed successfully.

Total execution time: 00:00:00.0013390

(1 row affected)

Total execution time: 00:00:00.0101498

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II

Вообще, вариантов решения задачи много. Но не все они оптимальны;)

Например, порой начинающие программисты для решения первой задачи создают что-то вроде этого:

In [33]:
use tempdb
go

declare @ClubId     int,
        @ClubCity   nvarchar(30), 
        @ClubName   nvarchar(100),
        @CategoryId int

set @ClubId = (select SwimmingClubId from dbo.SwimmingClub where [Name] like N'%Янтарь%')

set @ClubCity = (select City from dbo.SwimmingClub where [Name] like N'%Янтарь%')
set @ClubName = (select [Name] from dbo.SwimmingClub where [Name] like N'%Янтарь%')

set @CategoryId = (select CategoryId from dbo.Category where [Name] = N'II')

select SwimmerID, FirstName, LastName, YearOfBirth, Gender, 
       @ClubName Club, @ClubCity City, N'II' Category
from dbo.Swimmer 
where SwimmingClubID = @ClubId and CategoryId = @CategoryId


Commands completed successfully.

Total execution time: 00:00:00.0011336

(1 row affected)

Total execution time: 00:00:00.0046176

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II

<b>Пример #2.</b> Вывести спортсменов из клуба Янтарь с теми же атрибутами что и выше, но без требования иметь II спортивный разряд.

Используя старую нотацию:

In [23]:
use tempdb
go

--это код с багом! в случае если у спортсмена нет разряда, запись о нем не выводится
select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, c.[Name] Category
from dbo.SwimmingClub sc, dbo.Swimmer s, dbo.Category c
where sc.[Name] like N'%Янтарь%' 
      and sc.SwimmingClubID = s.SwimmingClubID
      and s.CategoryID     = c.CategoryID      

--подправленный код
select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, 
       (select c.[Name] from dbo.Category c where c.CategoryID = s.CategoryID) Category
from dbo.SwimmingClub sc, dbo.Swimmer s
where sc.[Name] like N'%Янтарь%' 
      and sc.SwimmingClubID = s.SwimmingClubID         


Commands completed successfully.

Total execution time: 00:00:00.0009234

(2 rows affected)

(3 rows affected)

Total execution time: 00:00:00.0084284

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 2 Анна Высоцкая 2007 ж ДЮСШ Янтарь Minsk кмс 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 2 Анна Высоцкая 2007 ж ДЮСШ Янтарь Minsk кмс 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II 11 Алексей Рылько 2005 м ДЮСШ Янтарь Minsk NULL

Используя новую нотацию:

In [24]:
use tempdb
go

select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, c.[Name] Category
from dbo.SwimmingClub sc
inner join dbo.Swimmer s  on s.SwimmingClubID = sc.SwimmingClubID
left join dbo.Category c on s.CategoryID     = c.CategoryID
where sc.[Name] like N'%Янтарь%' 

Commands completed successfully.

Total execution time: 00:00:00.0013412

(3 rows affected)

Total execution time: 00:00:00.0090951

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 2 Анна Высоцкая 2007 ж ДЮСШ Янтарь Minsk кмс 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II 11 Алексей Рылько 2005 м ДЮСШ Янтарь Minsk NULL

Вариант решения задачи с outer apply:

In [27]:
use tempdb
go

select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, c.[Name] Category
from dbo.SwimmingClub sc
inner join dbo.Swimmer s  on s.SwimmingClubID = sc.SwimmingClubID
outer apply (select [Name] from dbo.Category c where c.CategoryID = s.CategoryId) c 
where sc.[Name] like N'%Янтарь%' 

Commands completed successfully.

Total execution time: 00:00:00.0016745

(3 rows affected)

Total execution time: 00:00:00.0052674

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 2 Анна Высоцкая 2007 ж ДЮСШ Янтарь Minsk кмс 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk II 11 Алексей Рылько 2005 м ДЮСШ Янтарь Minsk NULL

Вариант решения задачи с пользовательской скалярной функцией:

In [29]:
use tempdb
go

create or alter function dbo.fn_GetCategoryName(@CategoryID int) returns nvarchar
as
begin
  return (select [Name] from dbo.Category where CategoryId = @CategoryID)
end
go

select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
       sc.[Name] Club, sc.City, dbo.fn_GetCategoryName(s.CategoryId) Category
from dbo.SwimmingClub sc
inner join dbo.Swimmer s  on s.SwimmingClubID = sc.SwimmingClubID
where sc.[Name] like N'%Янтарь%' 


Commands completed successfully.

Total execution time: 00:00:00.0013008

Commands completed successfully.

Total execution time: 00:00:00.0033548

(3 rows affected)

Total execution time: 00:00:00.0265345

SwimmerID FirstName LastName YearOfBirth Gender Club City Category 2 Анна Высоцкая 2007 ж ДЮСШ Янтарь Minsk к 6 Никита Клюй 2005 м ДЮСШ Янтарь Minsk I 11 Алексей Рылько 2005 м ДЮСШ Янтарь Minsk NULL

## Создавайте качественный код!